<a href="https://colab.research.google.com/github/Williamsjanthony15/Projects/blob/main/faceRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import os


In [ ]:
def faceRec(s):
  n = 0
  for i in s:
    n = n*10 + ord(i) - ord("0")
  return n

In [ ]:
mainFolders = ["test", "train"]
emotFolders = ['angry', 'sad', 'happy', 'fearful', 'surprised']
os.makedirs('data', exist_ok=True)
for mainFolders in mainFolders:
  os.makedirs(os.path.join("data",mainFolders), exist_ok=True)
  for emotFolders in emotFolders:
    os.makedirs(os.path.join('data', mainFolders, emotFolders), exist_ok=True)

angry = 0
sad = 0
happy = 0
fearful = 0
surprised = 0
angry_test = 0
sad_test = 0
happy_test = 0
fearful_test = 0
surprised_test = 0

In [ ]:
import zipfile
with zipfile.ZipFile("data.zip","r") as zip_ref:
    zip_ref.extractall("./")
df = pd.read_csv('./fer2013.csv')
mat = np.zeros((48,48), dtype=np.uint8)
print("here i am saving pictures")

for i in tqdm(range(len(df))):
  txt = df['pixels'][i]
  words = txt.split()

  for j in range(2304):
    xind = j // 48
    yind = j % 48
    # mat[xind][yind] = faceRec(words[i])

  img = Image.fromarray(mat)

  if i < 28709:
    if df['emotion'][i] == 0:
      img.save('./data/train/angry/im'+str(angry)+'.png')
      angry += 1
    if df['emotion'][i] == 0:
      img.save('./data/train/sad/im'+str(sad)+'.png')
      sad += 1
    if df['emotion'][i] == 0:
      img.save('./data/train/happy/im'+str(happy)+'.png')
      happy += 1
    if df['emotion'][i] == 0:
      img.save('./data/train/fearful/im'+str(fearful)+'.png')
      fearful += 1
    if df['emotion'][i] == 0:
      img.save('./data/train/surprised/im'+str(surprised)+'.png')
      surprised += 1

  else:
    if df['emotion'][i] == 0:
      img.save('./data/test/angry/im'+str(angry_test)+'.png')
      angry_test += 1
    elif df['emotion'][i] == 0:
      img.save('./data/test/sad/im'+str(sad_test)+'.png')
      sad_test += 1
    elif df['emotion'][i] == 0:
      img.save('./data/test/happy/im'+str(happy_test)+'.png')
      happy_test += 1
    elif df['emotion'][i] == 0:
      img.save('./data/test/fearful/im'+str(fearful_test)+'.png')
      fearful_test += 1
    elif df['emotion'][i] == 0:
      img.save('./data/test/surprised/im'+str(surprised_test)+'.png')
      surprised_test += 1

print('done son!')

here i am saving pictures


100%|██████████| 35887/35887 [00:32<00:00, 1093.63it/s]

done son!


In [ ]:
import numpy as np
import argparse

import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import argparse

def arguments():
  ap = argparse.ArgumentParser('--mode', help='train/display')
# ap.add_argument('--mode',help='train/display')
  mode = ap.parse_args().mode
  return mode

In [ ]:
train = 'data/train'
validate = 'data/test'

numTrain = 28709
numValidate = 7178
batch_size = 64
num_epoch = 50

datagenTrain = ImageDataGenerator(rescale=1./255)
datagenValidate = ImageDataGenerator(rescale=1./255)


trainGenerator = datagenTrain.flow_from_directory(
    train,
    target_size=(48, 48),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical"
    )

validateGenerator = datagenValidate.flow_from_directory(
    validate,
    target_size=(48, 48),
    batch_size = batch_size,
    color_mode = "grayscale",
    class_mode="categorical"
)

Found 31555 images belonging to 14 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [ ]:
if arguments == "train":
  model.compile(loss='categorical_crossentrophy', optimize=Adam(lr=0.0001, decay=ie-6), metrics=['accuracy'])
  model_info = model.fit_generator(
      train_generator,
      steps_per_epoch=numTrain // batch,
      epochs = num_epoch,
      validation_data = datagenValidate,
      validation_steps = numValidate // batch
  )
  plot_model_history(model_info)
  model.save_weights('model.h5')

elif arguments == "display":
    model.load_weights('model.h5')
    cv2.ocl.setUseOpenCL(False)
    emotionDict = {0: "Angry", 1: "Sad", 2: "Happy", 3: "Fearful", 4: "Surprised"}

    cap = cv2.VideoCapture(0)
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      fasecasc = cv2.CasadeClassifier('haarcascade_frontalface_defualt.xml')
      gray = cv2.cvtColor(frame, cv2.Color_BGR2GRAY)
      faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors = 5)

      for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame, emotionDict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

      cv2.imshow('Video', cv2.resize(frame,(1600, 960),interpolation = cv2.INTER_CUBIC))
      if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    cap.release()
    cv2.destroyAllWindows()